In [1]:
import os,sys
from src.ImageClassifier.Constants import *
from src.ImageClassifier.Exception import CustomException
from src.ImageClassifier.loggers import logger
from src.ImageClassifier.Utils import Save_Model,write_json_file,Create_Directory,read_yaml_file
from datetime import datetime
from pathlib import Path

## Evaluating the model of testing data

In [2]:
from dataclasses import dataclass

In [3]:
#step3) updating the entity file 
#entity file is nothing but we r defining the parameter as class variable which is used in this evaluating model stage

@dataclass
class EvaluatingModelConfig():
    trained_model_path:Path
    training_data_dir:Path #path where image dataset contain
    all_param:dict
    batch_size:int
    input_shape:list

## step4) update the configuration manager file

In [4]:
#in this file we read the yaml file and assign value to 
#entity class variable through yaml-->and rtn the functn

class ConfigurationManager():
    def __init__(self,config_filepath=CONFIG_FILEPATH,param_filepath=PARAM_FILEPATH):
        self.config = read_yaml_file(config_filepath)
        self.param = read_yaml_file(param_filepath)

        #creating artifacts directory
        Create_Directory([self.config.artifacts_root])

    def get_evaluation_config(self) ->EvaluatingModelConfig:
        #local variable
        eval = self.config.prepare_training

        #creating an object of EvaluatingModelConfig class
        eval_model_config = EvaluatingModelConfig(
            trained_model_path=eval.trained_model_path,
            training_data_dir=eval.training_data_dir,
            all_param=self.param,
            batch_size=self.param.batch_size,
            input_shape=self.param.input_shape
        )
        return eval_model_config

## step5)update the compoenent file

In [5]:
import tensorflow

In [6]:
#In this component file we create a class and initializing the instance variable to entity class variable
#and according to task writing the code

class EvaluatingModel():
    def __init__(self,eval:EvaluatingModelConfig):
        self.eval = eval

    #first loading the trained model
    def trained_model_load(self):
        self.model = tensorflow.keras.models.load_model(
            filepath=Path(self.eval.trained_model_path)
        )

    #2nd method creating ImageDatagenerator for testing data!!
    def _test_image_generator(self):
        #creating keyword argument for test datageenerator
        datagenerator_kwargs = dict(
            rescale=1.0/255,
            validation_split=0.30
        )

        #dataflow directory keyword argument
        dataflow_kwargs = dict(
            target_size = self.eval.input_shape[:-1],
            interpolation = "bilinear",
            batch_size = self.eval.batch_size,
        )

        #creating testdata generator !!
        test_generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        #now passing this test_generator to flow from imagae dtaset directory
        self.test_datagenerator = test_generator.flow_from_directory(
            directory = self.eval.training_data_dir,
            shuffle=False,
            **dataflow_kwargs,
            subset="validation"

        )

    #now evaluating the model
    def _evaluating_model(self):
        self.trained_model_load() #rtn model in same class
        self._test_image_generator() #rtn test dataset in same class
        evaluation_results  = self.model.evaluate(self.test_datagenerator)
        return evaluation_results

    #saving the score of model in json!!
    def _saving_score(self):
        result = self._evaluating_model()
        fileapth = "score.json"
        data = {"loss":result[0],"accuracy":result[1]}
        write_json_file(filepath=Path(fileapth),data=data)





In [7]:
os.chdir('../')
%pwd

'd:\\DL MODEL\\ImageClassificationModel'

## step6) updating pipeline

In [8]:
try:
    cm = ConfigurationManager()
    eval_config = cm.get_evaluation_config()

    em = EvaluatingModel(eval=eval_config)

    em.trained_model_load()

    em._test_image_generator()

    em._evaluating_model()

    em._saving_score()

    
except Exception as e:
    raise CustomException(e,sys)

[2024-09-13 00:47:32,098]-INFO-66-Reading the YAML file config\config.yaml
[2024-09-13 00:47:32,128]-INFO-69-YAML file read successfully: config\config.yaml
[2024-09-13 00:47:32,163]-INFO-66-Reading the YAML file params.yaml
[2024-09-13 00:47:32,167]-INFO-69-YAML file read successfully: params.yaml
[2024-09-13 00:47:32,219]-INFO-80-Creating Directory
Found 13 images belonging to 2 classes.
Found 13 images belonging to 2 classes.
3/3 [==============================] - 5s 922ms/step - loss: 0.6421 - accuracy: 0.6154
Found 13 images belonging to 2 classes.
3/3 [==============================] - 3s 933ms/step - loss: 0.6421 - accuracy: 0.6154
Data successfully written to score.json
